In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from google.colab import drive
from numpy.random import seed

In [2]:
drive.mount('/gdrive')
%cd /gdrive/MyDrive/F21DL_CW2/Data/CSV

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/F21DL_CW2/Data/CSV


In [3]:
X_train = pd.read_csv('x_train_gr_smpl.csv')
X_test = pd.read_csv('x_test_gr_smpl.csv')

X = pd.concat([X_train, X_test]).reset_index().drop(columns = ['index'])

y_train = pd.read_csv('y_train_smpl.csv')
y_test = pd.read_csv('y_test_smpl.csv')

y = pd.concat([y_train, y_test]).reset_index().drop(columns = ['index'])

X_train_4000, X_test_4000, y_train_4000, y_test_4000 = train_test_split(X, y, test_size=0.55, random_state=42, stratify = y)
X_train_9000, X_test_9000, y_train_9000, y_test_9000 = train_test_split(X, y, test_size=0.95, random_state=42, stratify = y)

X_train_4000 = X_train_4000.reset_index().drop(columns = ['index'])
X_test_4000 = X_test_4000.reset_index().drop(columns = ['index'])
y_train_4000 = y_train_4000.reset_index().drop(columns = ['index'])
y_test_4000 = y_test_4000.reset_index().drop(columns = ['index'])

X_train_9000 = X_train_9000.reset_index().drop(columns = ['index'])
X_test_9000 = X_test_9000.reset_index().drop(columns = ['index'])
y_train_9000 = y_train_9000.reset_index().drop(columns = ['index'])
y_test_9000 = y_test_9000.reset_index().drop(columns = ['index'])

X_train_tensor = tf.reshape(np.array(X_train), shape = (9690, 48, 48, 1))
X_test_tensor = tf.reshape(np.array(X_test), shape = (3090, 48, 48, 1))
X_train_4000_tensor = tf.reshape(np.array(X_train_4000), shape = (5750, 48, 48, 1))
X_test_4000_tensor = tf.reshape(np.array(X_test_4000), shape = (7030, 48, 48, 1))
X_train_9000_tensor = tf.reshape(np.array(X_train_9000), shape = (639, 48, 48, 1))
X_test_9000_tensor = tf.reshape(np.array(X_test_9000), shape = (12141, 48, 48, 1))

y_train_0 = pd.read_csv('y_train_smpl_0.csv')
y_test_0 = pd.read_csv('y_test_smpl_0.csv')

y_train_1 = pd.read_csv('y_train_smpl_1.csv')
y_test_1 = pd.read_csv('y_test_smpl_1.csv')

y_train_2 = pd.read_csv('y_train_smpl_2.csv')
y_test_2 = pd.read_csv('y_test_smpl_2.csv')

y_train_3 = pd.read_csv('y_train_smpl_3.csv')
y_test_3 = pd.read_csv('y_test_smpl_3.csv')

y_train_4 = pd.read_csv('y_train_smpl_4.csv')
y_test_4 = pd.read_csv('y_test_smpl_4.csv')

y_train_5 = pd.read_csv('y_train_smpl_5.csv')
y_test_5 = pd.read_csv('y_test_smpl_5.csv')

y_train_6 = pd.read_csv('y_train_smpl_6.csv')
y_test_6 = pd.read_csv('y_test_smpl_6.csv')

y_train_7 = pd.read_csv('y_train_smpl_7.csv')
y_test_7 = pd.read_csv('y_test_smpl_7.csv')

y_train_8 = pd.read_csv('y_train_smpl_8.csv')
y_test_8 = pd.read_csv('y_test_smpl_8.csv')

y_train_9 = pd.read_csv('y_train_smpl_9.csv')
y_test_9 = pd.read_csv('y_test_smpl_9.csv')

In [24]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state = 42)
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, min_delta = 0.0001, mode = 'min', restore_best_weights = True)
seed(42)

In [5]:
beta1 = [0.9, 0.09, 9]
beta2 = [0.999, 0.0999, 9.99]
lr = [0.001, 0.01, 0.1]

conv_kernels = [(4, 4), (3, 3)]
mp_kernels = [(3, 3), (2, 2)]

num_feature_maps = [64, 128, 256]

flattened_layers = [[512, 128], [1024, 512, 128], [2048, 1024, 512, 128]]

In [6]:
params = list()
acc = list()
class_reports = list()
%cd Data_from_cnn_tuning/testing_original/

/gdrive/MyDrive/F21DL_CW2/Data/CSV/Data_from_cnn_tuning/testing_original


In [7]:
for b1 in beta1:
  for b2 in beta2:
    for r in lr:
      adm = tf.keras.optimizers.Adam(learning_rate=r, beta_1=b1, beta_2=b2)
      for ck in range(len(conv_kernels)):
        for mk in range(len(mp_kernels)):
          for nf1 in num_feature_maps:
            for nf2 in num_feature_maps:
              model = tf.keras.Sequential([tf.keras.layers.Conv2D(nf1, conv_kernels[ck], activation='relu', input_shape=(48, 48, 1))])
              model.add(tf.keras.layers.MaxPooling2D(mp_kernels[mk]))
              model.add(tf.keras.layers.Conv2D(nf2, conv_kernels[ck], activation='relu'))
              model.add(tf.keras.layers.MaxPooling2D(mp_kernels[mk]))
              model.add(tf.keras.layers.Flatten())
              for l in range(len(flattened_layers)):
                for n in flattened_layers[l]:
                  model.add(tf.keras.layers.Dense(n, activation='relu'))
                model.add(tf.keras.layers.Dense(10, activation='softmax'))

                model.compile(optimizer=adm,
                              loss='sparse_categorical_crossentropy',
                              metrics=['accuracy'])

                params.append([b1, b2, r, ck, mk, nf1, nf2, l])
                print(params[-1])

                model.fit(x=X_train_tensor,y=y_train,epochs=300, callbacks = [es], verbose = 0)

                _, accu = model.evaluate(X_test_tensor, y_test, verbose = 0)

                acc.append(accu)

                y_pred = model.predict(X_test_tensor, verbose = 0)
                y_pred_bool = np.argmax(y_pred, axis=1)
                cr = pd.DataFrame(classification_report(y_test, y_pred_bool, output_dict=True)).T
                auc_p_c = list()

                for i in range(10):
                  fpr, tpr, _ = roc_curve(np.array(y_test), y_pred_bool, pos_label=i)
                  auc_p_c.append(auc(fpr, tpr))

                cr['auc'] = auc_p_c + [0, 0, 0]
                cr.loc['weighted avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc'] * cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'support']).sum() / cr.loc['macro avg', 'support']
                cr.loc['macro avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc']).mean()
                class_reports.append(cr)

                params.append([b1, b2, r, ck, mk, nf1, nf2, l])
                print(accu)
                cr.to_csv('original_b1_' + str(b1) + '_b2_' + str(b2) + '_lr_' + str(r) + '_ck_' + str(ck) + '_mk_' + str(mk) + '_nf1_' + str(nf1) + '_nf2_' + str(nf2) + '_l_' + str(l) + '.csv')

[0.9, 0.999, 0.001, 0, 0, 64, 64, 0]
0.9456310868263245
[0.9, 0.999, 0.001, 0, 0, 64, 64, 1]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.2330097109079361
[0.9, 0.999, 0.001, 0, 0, 64, 64, 2]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.24271844327449799
[0.9, 0.999, 0.001, 0, 0, 64, 128, 0]
0.9346278309822083
[0.9, 0.999, 0.001, 0, 0, 64, 128, 1]
0.9226537346839905
[0.9, 0.999, 0.001, 0, 0, 64, 128, 2]
0.9032362699508667
[0.9, 0.999, 0.001, 0, 0, 64, 256, 0]
0.9498381614685059
[0.9, 0.999, 0.001, 0, 0, 64, 256, 1]
0.940453052520752
[0.9, 0.999, 0.001, 0, 0, 64, 256, 2]


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.24271844327449799
[0.9, 0.999, 0.1, 1, 1, 256, 256, 2]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.2330097109079361
[0.9, 0.0999, 0.001, 0, 0, 64, 64, 0]


TypeError: ignored

In [20]:
params_csv = pd.DataFrame(params)
params_csv.columns = ['b1', 'b2', 'lr', 'ck', 'mk', 'nf1', 'nf2', 'l']
acc_csv = pd.DataFrame(acc)
acc_csv.columns = ['acc']

,acc
0,0.945631
1,0.233010
2,0.242718
3,0.934628
4,0.922654
...,...
319,0.233010
320,0.233010
321,0.233010
322,0.242718


In [22]:
params_csv.to_csv('params.csv')
acc_csv.to_csv('acc.csv')

In [ ]:
params_4000 = list()
acc_4000 = list()
class_reports_4000 = list()
%cd ../testing_4000/

In [ ]:
for b1 in beta1:
  for b2 in beta2:
    for r in lr:
      adm = tf.keras.optimizers.Adam(learning_rate=r, beta_1=b1, beta_2=b2)
      for ck in range(len(conv_kernels)):
        for mk in range(len(mp_kernels)):
          for nf1 in num_feature_maps:
            for nf2 in num_feature_maps:
              model = tf.keras.Sequential([tf.keras.layers.Conv2D(nf1, conv_kernels[ck], activation='relu', input_shape=(48, 48, 1))])
              model.add(tf.keras.layers.MaxPooling2D(mp_kernels[mk]))
              model.add(tf.keras.layers.Conv2D(nf2, conv_kernels[ck], activation='relu'))
              model.add(tf.keras.layers.MaxPooling2D(mp_kernels[mk]))
              model.add(tf.keras.layers.Flatten())
              for l in range(len(flattened_layers)):
                for n in flattened_layers[l]:
                  model.add(tf.keras.layers.Dense(n, activation='relu'))
                model.add(tf.keras.layers.Dense(10, activation='softmax'))

                model.compile(optimizer=adm,
                              loss='sparse_categorical_crossentropy',
                              metrics=['accuracy'])
                
                params_4000.append([b1, b2, r, ck, mk, nf1, nf2, l])
                print(params_4000[-1])

                model.fit(x=X_train_4000_tensor,y=y_train_4000,epochs=300, callbacks = [es], verbose = 0)

                _, accu = model.evaluate(X_test_4000_tensor, y_test_4000, verbose = 0)

                acc_4000.append(accu)

                y_pred = model.predict(X_test_4000_tensor, verbose = 0)
                y_pred_bool = np.argmax(y_pred, axis=1)
                cr = pd.DataFrame(classification_report(y_test_4000, y_pred_bool, output_dict=True)).T
                auc_p_c = list()

                for i in range(10):
                  fpr, tpr, _ = roc_curve(np.array(y_test_4000), y_pred_bool, pos_label=i)
                  auc_p_c.append(auc(fpr, tpr))
                  
                cr['auc'] = auc_p_c + [0, 0, 0]
                cr.loc['weighted avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc'] * cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'support']).sum() / cr.loc['macro avg', 'support']
                cr.loc['macro avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc']).mean()
                class_reports_4000.append(cr)

                print(accu)
                
                cr.to_csv('testing_4000_b1_' + str(b1) + '_b2_' + str(b2) + '_lr_' + str(r) + '_ck_' + str(ck) + '_mk_' + str(mk) + '_nf1_' + str(nf1) + '_nf2_' + str(nf2) + '_l_' + str(l) + '.csv')

In [25]:
params_9000 = list()
acc_9000 = list()
class_reports_9000 = list()
%cd ../testing_9000/

/gdrive/My Drive/F21DL_CW2/Data/CSV/Data_from_cnn_tuning/testing_9000


In [26]:
for b1 in beta1:
  for b2 in beta2:
    for r in lr:
      adm = tf.keras.optimizers.Adam(learning_rate=r, beta_1=b1, beta_2=b2)
      for ck in range(len(conv_kernels)):
        for mk in range(len(mp_kernels)):
          for nf1 in num_feature_maps:
            for nf2 in num_feature_maps:
              model = tf.keras.Sequential([tf.keras.layers.Conv2D(nf1, conv_kernels[ck], activation='relu', input_shape=(48, 48, 1))])
              model.add(tf.keras.layers.MaxPooling2D(mp_kernels[mk]))
              model.add(tf.keras.layers.Conv2D(nf2, conv_kernels[ck], activation='relu'))
              model.add(tf.keras.layers.MaxPooling2D(mp_kernels[mk]))
              model.add(tf.keras.layers.Flatten())
              for l in range(len(flattened_layers)):
                for n in flattened_layers[l]:
                  model.add(tf.keras.layers.Dense(n, activation='relu'))
                model.add(tf.keras.layers.Dense(10, activation='softmax'))

                model.compile(optimizer=adm,
                              loss='sparse_categorical_crossentropy',
                              metrics=['accuracy'])
                
                params_9000.append([b1, b2, r, ck, mk, nf1, nf2, l])
                print(params_9000[-1])

                model.fit(x=X_train_9000_tensor,y=y_train_9000,epochs=10, callbacks = [es], verbose = 0)

                _, accu = model.evaluate(X_test_9000_tensor, y_test_9000, verbose = 0)

                acc_9000.append(accu)

                y_pred = model.predict(X_test_9000_tensor, verbose = 0)
                y_pred_bool = np.argmax(y_pred, axis=1)
                cr = pd.DataFrame(classification_report(y_test_9000, y_pred_bool, output_dict=True)).T
                auc_p_c = list()

                for i in range(10):
                  fpr, tpr, _ = roc_curve(np.array(y_test_9000), y_pred_bool, pos_label=i)
                  auc_p_c.append(auc(fpr, tpr))
                  
                cr['auc'] = auc_p_c + [0, 0, 0]
                cr.loc['weighted avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc'] * cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'support']).sum() / cr.loc['macro avg', 'support']
                cr.loc['macro avg', 'auc'] = (cr.loc[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], 'auc']).mean()
                class_reports_9000.append(cr)

                
                print(accu)
                
                cr.to_csv('testing_4000_b1_' + str(b1) + '_b2_' + str(b2) + '_lr_' + str(r) + '_ck_' + str(ck) + '_mk_' + str(mk) + '_nf1_' + str(nf1) + '_nf2_' + str(nf2) + '_l_' + str(l) + '.csv')

[0.9, 0.999, 0.001, 0, 0, 64, 64, 0]
0.8082530498504639
[0.9, 0.999, 0.001, 0, 0, 64, 64, 1]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.808664858341217
[0.9, 0.999, 0.001, 0, 0, 64, 64, 2]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6174944639205933
[0.9, 0.999, 0.001, 0, 0, 64, 128, 0]
0.7254756689071655
[0.9, 0.999, 0.001, 0, 0, 64, 128, 1]
0.8001812100410461
[0.9, 0.999, 0.001, 0, 0, 64, 128, 2]


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.23474177718162537
[0.9, 0.0999, 0.1, 1, 1, 256, 256, 2]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.2065727710723877
[0.9, 9.99, 0.001, 0, 0, 64, 64, 0]


TypeError: ignored

In [ ]:
params_csv = pd.DataFrame(params_9000)
params_csv.columns = ['b1', 'b2', 'lr', 'ck', 'mk', 'nf1', 'nf2', 'l']
acc_csv = pd.DataFrame(acc_9000)
acc_csv.columns = ['acc']

In [ ]:
params_csv.to_csv('params.csv')
acc_csv.to_csv('acc.csv')

In [ ]:
print(min(acc))
print(params[acc.index(min(acc))])

In [ ]:
print(min(acc_4000))
print(params_4000[acc_4000.index(min(acc_4000))])

In [ ]:
print(min(acc_9000))
print(params_9000[acc_4000.index(min(acc_9000))])

In [ ]:
model.summary()

In [ ]:
print(max(acc))
print(params[acc.index(max(acc))])

In [ ]:
print(max(acc_4000))
print(params_4000[acc_4000.index(max(acc_4000))])

In [ ]:
print(max(acc_9000))
print(params_9000[acc_4000.index(max(acc_9000))])